## Document Loading

In [1]:
# ! pip install langchain

In [2]:
# ! pip install -U langchain-community

In [3]:
# pip install dotenv

In [4]:
# !conda install -c conda-forge ffmpeg -y

In [5]:
# !pip install openai-whisper

In [6]:
import sys
import subprocess

# Install in the current Python environment
subprocess.check_call([sys.executable, "-m", "pip", "install", "pydub"])

0

In [7]:
import whisper
whisper_model = whisper.load_model("base")  # options: tiny, base, small, medium, large

In [8]:
import os
import openai

sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# openai.api_key = os.environ['OPENAI_API_KEY']

In [9]:
# pip install pypdf

In [10]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("MachineLearning-Lecture01.pdf")
pages = loader.load()

In [11]:
len(pages)

22

In [12]:
page = pages[0]

In [13]:
print(page.page_content[0:500])

MachineLearning-Lecture01  
Instructor (Andrew Ng): Okay. Good morning. Welcome to CS229, the machine 
learning class. So what I wanna do today is just spend a little time going over the logistics 
of the class, and then we'll start to talk a bit about machine learning.  
By way of introduction, my name's Andrew Ng and I'll be instructor for this class. And so 
I personally work in machine learning, and I've worked on it for about 15 years now, and 
I actually think that machine learning is the 


In [14]:
page.metadata

{'producer': 'Acrobat Distiller 8.1.0 (Windows)',
 'creator': 'PScript5.dll Version 5.2.2',
 'creationdate': '2008-07-11T11:25:23-07:00',
 'author': '',
 'moddate': '2008-07-11T11:25:23-07:00',
 'title': '',
 'source': 'MachineLearning-Lecture01.pdf',
 'total_pages': 22,
 'page': 0,
 'page_label': '1'}

In [15]:
! pip install yt_dlp
! pip install pydub

In [16]:
pip install pydub

Note: you may need to restart the kernel to use updated packages.


In [17]:
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader
from langchain_community.document_loaders.blob_loaders import FileSystemBlobLoader

In [18]:
import yt_dlp

In [19]:
url="https://www.youtube.com/watch?v=jGwO_UgTS7I"
save_dir="docs/youtube/"
os.makedirs(save_dir, exist_ok=True)

# loader = GenericLoader(
#     YoutubeAudioLoader([url],save_dir),  # fetch from youtube
#     #FileSystemBlobLoader(save_dir, glob="*.m4a"),   #fetch locally
#     OpenAIWhisperParser()
# )
# docs = loader.load()

In [20]:
# Download audio
ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl': os.path.join(save_dir, '%(title)s.%(ext)s'),
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
}

In [21]:
print("Downloading audio...")
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

[youtube] Extracting URL: https://www.youtube.com/watch?v=jGwO_UgTS7I
[youtube] jGwO_UgTS7I: Downloading webpage
[youtube] jGwO_UgTS7I: Downloading tv client config
[youtube] jGwO_UgTS7I: Downloading tv player API JSON
[youtube] jGwO_UgTS7I: Downloading web safari player API JSON
[youtube] jGwO_UgTS7I: Downloading m3u8 information
[info] jGwO_UgTS7I: Downloading 1 format(s): 251
[download] Sleeping 4.00 seconds as required by the site...
[download] Destination: docs/youtube/Stanford CS229： Machine Learning Course, Lecture 1 - Andrew Ng (Autumn 2018).webm
[download] 100% of   51.57MiB in 00:00:09 at 5.63MiB/s     
[ExtractAudio] Destination: docs/youtube/Stanford CS229： Machine Learning Course, Lecture 1 - Andrew Ng (Autumn 2018).mp3
Deleting original file docs/youtube/Stanford CS229： Machine Learning Course, Lecture 1 - Andrew Ng (Autumn 2018).webm (pass -k to keep)


In [22]:
from langchain.schema import Document

In [23]:
# Find the downloaded audio file
audio_files = [f for f in os.listdir(save_dir) if f.endswith(('.mp3', '.m4a', '.wav'))]
print(f"Found audio files: {audio_files}")

Found audio files: ['Stanford CS229： Machine Learning Course, Lecture 1 - Andrew Ng (Autumn 2018).m4a', 'Stanford CS229： Machine Learning Course, Lecture 1 - Andrew Ng (Autumn 2018).mp3']


In [ ]:
# Transcribe
docs = []
for audio_file in audio_files:
    file_path = os.path.join(save_dir, audio_file)
    print(f"Transcribing {audio_file}...")
    result = whisper_model.transcribe(file_path)
    docs.append(Document(
        page_content=result["text"],
        metadata={"source": url, "file": audio_file}
    ))

Transcribing Stanford CS229： Machine Learning Course, Lecture 1 - Andrew Ng (Autumn 2018).m4a...


/opt/anaconda3/envs/myenv/lib/python3.13/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcribing Stanford CS229： Machine Learning Course, Lecture 1 - Andrew Ng (Autumn 2018).mp3...


/opt/anaconda3/envs/myenv/lib/python3.13/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [25]:
print(f"Number of docs: {len(docs)}")
if len(docs) > 0:
    print(docs[0].page_content[0:500])

Number of docs: 2
 Welcome to CS229 Machine Learning. Some of you know that this class is for this Stanford for long time and this is often the class that I most look for to teaching each year because this is where we've helped I think several generations of Stanford students become experts in machine learning. Got built many of their products and services and startups that I'm sure many of you will pray all of you are using today. So what I want to do today was spend some time talking over logistics and then spe


In [26]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://github.com/Rnamrata/image_enhancement_for_social_robots/blob/master/README.md")

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [27]:
docs = loader.load()

In [28]:
print(f"Number of docs: {len(docs)}")

Number of docs: 1


In [29]:
if len(docs) > 0:
    print(docs[0].page_content[0:1000])




















































































image_enhancement_for_social_robots/README.md at master · Rnamrata/image_enhancement_for_social_robots · GitHub
















































Skip to content













Navigation Menu

Toggle navigation




 













            Sign in
          


 


Appearance settings











        Platform
        















          GitHub Copilot

        

        Write better code wit


## Document Splitting

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [62]:
chunk_size =26
chunk_overlap = 4

In [64]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

In [66]:
text1 = 'abcdefghijklmnopqrstuvwxyz'

In [68]:
r_splitter.split_text(text1)

['abcdefghijklmnopqrstuvwxyz']

In [70]:
text2 = 'abcdefghijklmnopqrstuvwxyzabcdefg'

In [72]:
r_splitter.split_text(text2)

['abcdefghijklmnopqrstuvwxyz', 'wxyzabcdefg']

In [74]:
text3 = "a b c d e f g h i j k l m n o p q r s t u v w x y z"

In [76]:
r_splitter.split_text(text3)

['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

In [78]:
c_splitter.split_text(text3)

['a b c d e f g h i j k l m n o p q r s t u v w x y z']

In [80]:
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separator = ' '
)
c_splitter.split_text(text3)

['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

### Recursive Splitting Details

In [82]:
some_text = """When writing documents, writers will use document structure to group content. \
This can convey to the reader, which idea's are related. For example, closely related ideas \
are in sentances. Similar ideas are in paragraphs. Paragraphs form a document. \n\n  \
Paragraphs are often delimited with a carriage return or two carriage returns. \
Carriage returns are the "backslash n" you see embedded in this string. \
Sentences have a period at the end, but also, have a space.\
and words are separated by space."""

In [84]:
len(some_text)

496

In [86]:
c_splitter = CharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0,
    separator = ' '
)
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0, 
    separators=["\n\n", "\n", " ", ""]
)

In [88]:
c_splitter.split_text(some_text)

['When writing documents, writers will use document structure to group content. This can convey to the reader, which idea\'s are related. For example, closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document. \n\n Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this string. Sentences have a period at the end, but also,',
 'have a space.and words are separated by space.']

In [90]:
r_splitter.split_text(some_text)

["When writing documents, writers will use document structure to group content. This can convey to the reader, which idea's are related. For example, closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document.",
 'Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this string. Sentences have a period at the end, but also, have a space.and words are separated by space.']

In [92]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separators=["\n\n", "\n", "\. ", " ", ""]
)
r_splitter.split_text(some_text)

<>:4: SyntaxWarning: invalid escape sequence '\.'
<>:4: SyntaxWarning: invalid escape sequence '\.'
/var/folders/nb/77hjsc2j58s68phj_xx_yps80000gn/T/ipykernel_33422/158012430.py:4: SyntaxWarning: invalid escape sequence '\.'
  separators=["\n\n", "\n", "\. ", " ", ""]


["When writing documents, writers will use document structure to group content. This can convey to the reader, which idea's are related. For example,",
 'closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document.',
 'Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this',
 'string. Sentences have a period at the end, but also, have a space.and words are separated by space.']

In [94]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)
r_splitter.split_text(some_text)

<>:4: SyntaxWarning: invalid escape sequence '\.'
<>:4: SyntaxWarning: invalid escape sequence '\.'
/var/folders/nb/77hjsc2j58s68phj_xx_yps80000gn/T/ipykernel_33422/2945222903.py:4: SyntaxWarning: invalid escape sequence '\.'
  separators=["\n\n", "\n", "(?<=\. )", " ", ""]


["When writing documents, writers will use document structure to group content. This can convey to the reader, which idea's are related. For example,",
 'closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document.',
 'Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this',
 'string. Sentences have a period at the end, but also, have a space.and words are separated by space.']

In [98]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("MachineLearning-Lecture01.pdf")
pages = loader.load()

In [100]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)

In [102]:
docs = text_splitter.split_documents(pages)

In [104]:
len(docs)

78

In [106]:
len(pages)

22

In [160]:
docs[0]

Document(metadata={'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2008-07-11T11:25:23-07:00', 'author': '', 'moddate': '2008-07-11T11:25:23-07:00', 'title': '', 'source': 'MachineLearning-Lecture01.pdf', 'total_pages': 22, 'page': 0, 'page_label': '1'}, page_content='MachineLearning-Lecture01  \n')

In [162]:
pages[0]

Document(metadata={'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2008-07-11T11:25:23-07:00', 'author': '', 'moddate': '2008-07-11T11:25:23-07:00', 'title': '', 'source': 'MachineLearning-Lecture01.pdf', 'total_pages': 22, 'page': 0, 'page_label': '1'}, page_content='MachineLearning-Lecture01  \nInstructor (Andrew Ng): Okay. Good morning. Welcome to CS229, the machine \nlearning class. So what I wanna do today is just spend a little time going over the logistics \nof the class, and then we\'ll start to talk a bit about machine learning.  \nBy way of introduction, my name\'s Andrew Ng and I\'ll be instructor for this class. And so \nI personally work in machine learning, and I\'ve worked on it for about 15 years now, and \nI actually think that machine learning is the most exciting field of all the computer \nsciences. So I\'m actually always excited about teaching this class. Sometimes I actually \nthink that machine learning 

### Token Splitting

In [108]:
from langchain.text_splitter import TokenTextSplitter

In [112]:
text_splitter = TokenTextSplitter(chunk_size=10, chunk_overlap=0)

In [114]:
docs = text_splitter.split_documents(pages)

In [168]:
len(docs)

1481

In [166]:
docs[3].page_content

'learning class. So what I wanna do today is'

In [118]:
pages[0].metadata

{'producer': 'Acrobat Distiller 8.1.0 (Windows)',
 'creator': 'PScript5.dll Version 5.2.2',
 'creationdate': '2008-07-11T11:25:23-07:00',
 'author': '',
 'moddate': '2008-07-11T11:25:23-07:00',
 'title': '',
 'source': 'MachineLearning-Lecture01.pdf',
 'total_pages': 22,
 'page': 0,
 'page_label': '1'}

### Context Aware Splitting

In [142]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

In [122]:
markdown_document = """# Title\n\n \
## Chapter 1\n\n \
Hi this is Jim\n\n Hi this is Joe\n\n \
### Section \n\n \
Hi this is Lance \n\n 
## Chapter 2\n\n \
Hi this is Molly"""

In [124]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

In [126]:
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)
md_header_splits = markdown_splitter.split_text(markdown_document)

In [128]:
md_header_splits[0]

Document(metadata={'Header 1': 'Title', 'Header 2': 'Chapter 1'}, page_content='Hi this is Jim  \nHi this is Joe')

In [130]:
md_header_splits[1]

Document(metadata={'Header 1': 'Title', 'Header 2': 'Chapter 1', 'Header 3': 'Section'}, page_content='Hi this is Lance')

In [132]:
txt = ' '.join([d.page_content for d in docs])

In [172]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
]
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)

In [186]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

md_header_splits = markdown_splitter.split_text(txt)
splits = text_splitter.split_documents(md_header_splits)

In [188]:
len(splits)

70

In [190]:
splits[0]

Document(metadata={}, page_content="MachineLearning-Lecture01\nInstructor (Andrew Ng): Okay. Good morning . Welcome to CS229, the machine\nlearning class. So what I wanna do today is  just spend a little time going over the logistics\nof the class, and then we'll start  to talk a bit about machine learning.\nBy way of introduction, my name's Andrew  Ng and I'll be instructor for this class.  And so\nI personally work in machine learning , and I've worked on it for about 15  years now, and\nI actually think that  machine learning is the most exciting field of all the  computer\nsciences. So I'm actually  always excited about teaching this class. Sometimes I actually\nthink that machine learning is not only the  most exciting thing in computer science, but\nthe most exciting thing in all of human endeavor,  so maybe a little bias there.\nI also want to introduce the TAs, who  are all graduate students doing research in or\nrelated to the machine learning and all aspects of machine  learn

In [192]:
splits[0].page_content

"MachineLearning-Lecture01\nInstructor (Andrew Ng): Okay. Good morning . Welcome to CS229, the machine\nlearning class. So what I wanna do today is  just spend a little time going over the logistics\nof the class, and then we'll start  to talk a bit about machine learning.\nBy way of introduction, my name's Andrew  Ng and I'll be instructor for this class.  And so\nI personally work in machine learning , and I've worked on it for about 15  years now, and\nI actually think that  machine learning is the most exciting field of all the  computer\nsciences. So I'm actually  always excited about teaching this class. Sometimes I actually\nthink that machine learning is not only the  most exciting thing in computer science, but\nthe most exciting thing in all of human endeavor,  so maybe a little bias there.\nI also want to introduce the TAs, who  are all graduate students doing research in or\nrelated to the machine learning and all aspects of machine  learning. Paul Baumstarck"

In [194]:
splits[1].page_content

"related to the machine learning and all aspects of machine  learning. Paul Baumstarck\nworks  in machine learning and computer vision. Catie Chang  is actually a neuroscientist\nwho applies  machine learning algorithms to try to understand the human brain . Tom Do\nis another PhD student,  works in computational biology and in sort of the basic\nfundamentals of human learning. Z ico Kolter is the head TA — he's  head TA two years\nin a row now  — works in machine learning a nd applies them  to a bunch of robots. And\nDaniel  Ramage is — I guess he's not here   — Daniel applies l earning algorithms to\nproblems in natural language processing.\nSo you'll get to know the TAs and  me much better throughout this quarter, but just from\nthe sorts of things the TA's do , I hope you can already tell that machine learning  is a\nhighly interdisciplinary topic in which  just the TAs find learning algorithms to problems"

In [136]:
md_header_splits = markdown_splitter.split_text(txt)

In [157]:
len(md_header_splits)

1

In [138]:
md_header_splits[0]

Document(metadata={}, page_content='MachineLearning-Lecture01\nInstructor (Andrew Ng): Okay. Good morning . Welcome to CS229, the machine\nlearning class. So what I wanna do today is  just spend a little time going over the logistics\nof the class, and then we\'ll start  to talk a bit about machine learning.\nBy way of introduction, my name\'s Andrew  Ng and I\'ll be instructor for this class.  And so\nI personally work in machine learning , and I\'ve worked on it for about 15  years now, and\nI actually think that  machine learning is the most exciting field of all the  computer\nsciences. So I\'m actually  always excited about teaching this class. Sometimes I actually\nthink that machine learning is not only the  most exciting thing in computer science, but\nthe most exciting thing in all of human endeavor,  so maybe a little bias there.\nI also want to introduce the TAs, who  are all graduate students doing research in or\nrelated to the machine learning and all aspects of machine  

## Embeddings

In [155]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

/var/folders/nb/77hjsc2j58s68phj_xx_yps80000gn/T/ipykernel_33422/1742550774.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()


ValidationError: 1 validation error for OpenAIEmbeddings
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'model_kwargs': {}, 'cli...20, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error